# 🧪 Testes da API SPTrans - Tempo Real

Este notebook realiza testes completos da API SPTrans em tempo real.

## Objetivos
1. Validar autenticação e conectividade com a API
2. Testar todos os endpoints disponíveis
3. Analisar estrutura e qualidade dos dados retornados
4. Verificar performance e latência
5. Detectar anomalias e inconsistências
6. Documentar exemplos de resposta

---

In [ ]:
# Imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import time
from datetime import datetime, timedelta
from pathlib import Path
import warnings
from typing import Dict, List, Optional
import os
from dotenv import load_dotenv

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

# Carregar variáveis de ambiente
load_dotenv('../.env')

print("✅ Bibliotecas importadas com sucesso!")

## 1️⃣ Configuração da API

In [ ]:
# Configurações da API SPTrans
API_TOKEN = os.getenv('SPTRANS_API_TOKEN', 'YOUR_TOKEN_HERE')
BASE_URL = 'http://api.olhovivo.sptrans.com.br/v2.1'

# Verificar token
if API_TOKEN == 'YOUR_TOKEN_HERE':
    print("⚠️ ATENÇÃO: Configure seu token da API SPTrans no arquivo .env")
    print("\nPara obter um token:")
    print("1. Acesse: http://www.sptrans.com.br/desenvolvedores/")
    print("2. Cadastre-se e solicite um token de acesso")
    print("3. Adicione o token no arquivo .env: SPTRANS_API_TOKEN=seu_token_aqui")
else:
    print(f"✅ Token configurado: {API_TOKEN[:10]}...{API_TOKEN[-10:]}")
    print(f"✅ Base URL: {BASE_URL}")

In [ ]:
# Classe para interagir com a API
class SPTransAPITester:
    """Classe para testar a API SPTrans"""
    
    def __init__(self, token: str, base_url: str):
        self.token = token
        self.base_url = base_url
        self.session = requests.Session()
        self.authenticated = False
        self.test_results = []
    
    def authenticate(self) -> bool:
        """Autentica na API"""
        url = f"{self.base_url}/Login/Autenticar?token={self.token}"
        
        try:
            start_time = time.time()
            response = self.session.post(url)
            elapsed = time.time() - start_time
            
            self.authenticated = response.text.lower() == 'true'
            
            result = {
                'endpoint': 'Login/Autenticar',
                'success': self.authenticated,
                'status_code': response.status_code,
                'latency_ms': round(elapsed * 1000, 2),
                'response': response.text
            }
            self.test_results.append(result)
            
            return self.authenticated
        except Exception as e:
            print(f"❌ Erro na autenticação: {e}")
            return False
    
    def test_endpoint(self, endpoint: str, params: Optional[Dict] = None) -> Dict:
        """Testa um endpoint específico"""
        if not self.authenticated:
            print("⚠️ Não autenticado! Execute authenticate() primeiro.")
            return None
        
        url = f"{self.base_url}/{endpoint}"
        
        try:
            start_time = time.time()
            response = self.session.get(url, params=params)
            elapsed = time.time() - start_time
            
            try:
                data = response.json()
            except:
                data = response.text
            
            result = {
                'endpoint': endpoint,
                'success': response.status_code == 200,
                'status_code': response.status_code,
                'latency_ms': round(elapsed * 1000, 2),
                'data_type': type(data).__name__,
                'data_size': len(str(data)),
                'response': data
            }
            
            self.test_results.append(result)
            return result
            
        except Exception as e:
            print(f"❌ Erro ao testar {endpoint}: {e}")
            return {'endpoint': endpoint, 'success': False, 'error': str(e)}
    
    def get_summary(self) -> pd.DataFrame:
        """Retorna resumo dos testes"""
        df = pd.DataFrame(self.test_results)
        return df[['endpoint', 'success', 'status_code', 'latency_ms']]

# Inicializar tester
api_tester = SPTransAPITester(API_TOKEN, BASE_URL)
print("✅ API Tester inicializado!")

## 2️⃣ Teste de Autenticação

In [ ]:
print("🔐 Testando autenticação...\n")

auth_success = api_tester.authenticate()

if auth_success:
    print("✅ Autenticação bem-sucedida!")
    print(f"   Status: {api_tester.test_results[-1]['status_code']}")
    print(f"   Latência: {api_tester.test_results[-1]['latency_ms']} ms")
else:
    print("❌ Falha na autenticação!")
    print("\nVerifique:")
    print("  1. Token está correto")
    print("  2. Conexão com a internet")
    print("  3. API SPTrans está disponível")

## 3️⃣ Teste de Endpoints - Linhas

In [ ]:
print("🚌 Testando endpoints de LINHAS...\n")

# 1. Buscar linhas
print("1️⃣ Buscar Linhas (termo: 'Centro')")
result = api_tester.test_endpoint('Linha/Buscar', params={'termosBusca': 'Centro'})
if result and result['success']:
    linhas = result['response']
    print(f"   ✅ Encontradas {len(linhas)} linhas")
    print(f"   📊 Latência: {result['latency_ms']} ms")
    if linhas:
        print(f"\n   Exemplo de linha:")
        print(f"   {json.dumps(linhas[0], indent=2, ensure_ascii=False)}")
else:
    print("   ❌ Falha ao buscar linhas")

print("\n" + "="*60 + "\n")

# 2. Listar todas as linhas (se disponível)
print("2️⃣ Listar Todas as Linhas")
result = api_tester.test_endpoint('Linha/Listar')
if result and result['success']:
    print(f"   ✅ Total de linhas: {len(result['response'])}")
    print(f"   📊 Latência: {result['latency_ms']} ms")
else:
    print("   ℹ️ Endpoint não disponível ou sem dados")

## 4️⃣ Teste de Endpoints - Paradas

In [ ]:
print("🚏 Testando endpoints de PARADAS...\n")

# 1. Buscar paradas
print("1️⃣ Buscar Paradas (termo: 'Paulista')")
result = api_tester.test_endpoint('Parada/Buscar', params={'termosBusca': 'Paulista'})
if result and result['success']:
    paradas = result['response']
    print(f"   ✅ Encontradas {len(paradas)} paradas")
    print(f"   📊 Latência: {result['latency_ms']} ms")
    if paradas:
        print(f"\n   Exemplo de parada:")
        print(f"   {json.dumps(paradas[0], indent=2, ensure_ascii=False)}")
else:
    print("   ❌ Falha ao buscar paradas")

print("\n" + "="*60 + "\n")

# 2. Buscar paradas por corredor
print("2️⃣ Buscar Paradas por Corredor (código: 1)")
result = api_tester.test_endpoint('Parada/BuscarParadasPorCorredor', params={'codigoCorredor': 1})
if result and result['success']:
    print(f"   ✅ Paradas encontradas no corredor")
    print(f"   📊 Latência: {result['latency_ms']} ms")
else:
    print("   ℹ️ Endpoint não disponível ou sem dados")

## 5️⃣ Teste de Endpoints - Posições em Tempo Real

In [ ]:
print("📍 Testando endpoints de POSIÇÕES EM TEMPO REAL...\n")

# Buscar uma linha primeiro para testar posições
linhas_result = api_tester.test_endpoint('Linha/Buscar', params={'termosBusca': '8000'})

if linhas_result and linhas_result['success'] and linhas_result['response']:
    codigo_linha = linhas_result['response'][0].get('cl', None)
    
    if codigo_linha:
        print(f"1️⃣ Posições dos Veículos (Linha: {codigo_linha})")
        result = api_tester.test_endpoint('Posicao/Linha', params={'codigoLinha': codigo_linha})
        
        if result and result['success']:
            data = result['response']
            if isinstance(data, dict) and 'vs' in data:
                veiculos = data['vs']
                print(f"   ✅ {len(veiculos)} veículos ativos")
                print(f"   📊 Latência: {result['latency_ms']} ms")
                print(f"   🕐 Horário de referência: {data.get('hr', 'N/A')}")
                
                if veiculos:
                    print(f"\n   Exemplo de veículo:")
                    print(f"   {json.dumps(veiculos[0], indent=2, ensure_ascii=False)}")
                    
                    # Criar DataFrame para análise
                    df_veiculos = pd.DataFrame(veiculos)
                    print(f"\n   📊 Resumo estatístico:")
                    print(f"   - Veículos únicos: {df_veiculos['p'].nunique() if 'p' in df_veiculos.columns else 'N/A'}")
                    if 'py' in df_veiculos.columns and 'px' in df_veiculos.columns:
                        print(f"   - Lat média: {df_veiculos['py'].mean():.6f}")
                        print(f"   - Lon média: {df_veiculos['px'].mean():.6f}")
            else:
                print("   ℹ️ Nenhum veículo ativo no momento")
        else:
            print("   ❌ Falha ao buscar posições")
    else:
        print("   ⚠️ Não foi possível obter código da linha")
else:
    print("   ⚠️ Não foi possível buscar linhas para teste")

## 6️⃣ Teste de Endpoints - Previsão de Chegada

In [ ]:
print("⏰ Testando endpoints de PREVISÃO DE CHEGADA...\n")

# Buscar uma parada primeiro
paradas_result = api_tester.test_endpoint('Parada/Buscar', params={'termosBusca': 'Paulista'})

if paradas_result and paradas_result['success'] and paradas_result['response']:
    codigo_parada = paradas_result['response'][0].get('cp', None)
    
    if codigo_parada:
        print(f"1️⃣ Previsão de Chegada (Parada: {codigo_parada})")
        result = api_tester.test_endpoint('Previsao/Parada', params={'codigoParada': codigo_parada})
        
        if result and result['success']:
            data = result['response']
            if isinstance(data, dict):
                print(f"   ✅ Previsões obtidas")
                print(f"   📊 Latência: {result['latency_ms']} ms")
                print(f"   🕐 Horário: {data.get('hr', 'N/A')}")
                
                if 'p' in data and 'l' in data['p']:
                    linhas_previsao = data['p']['l']
                    print(f"   🚌 {len(linhas_previsao)} linhas com previsão")
                    
                    if linhas_previsao:
                        print(f"\n   Exemplo de previsão:")
                        print(f"   {json.dumps(linhas_previsao[0], indent=2, ensure_ascii=False)}")
        else:
            print("   ❌ Falha ao buscar previsões")
    else:
        print("   ⚠️ Não foi possível obter código da parada")
else:
    print("   ⚠️ Não foi possível buscar paradas para teste")

## 7️⃣ Teste de Endpoints - Corredores

In [ ]:
print("🛣️ Testando endpoints de CORREDORES...\n")

print("1️⃣ Listar Corredores")
result = api_tester.test_endpoint('Corredor')
if result and result['success']:
    corredores = result['response']
    print(f"   ✅ {len(corredores)} corredores encontrados")
    print(f"   📊 Latência: {result['latency_ms']} ms")
    
    if corredores:
        print(f"\n   Corredores disponíveis:")
        for corredor in corredores[:5]:  # Primeiros 5
            print(f"   - {corredor}")
else:
    print("   ℹ️ Endpoint não disponível ou sem dados")

## 8️⃣ Análise de Performance

In [ ]:
# Resumo dos testes
summary_df = api_tester.get_summary()

print("📊 RESUMO DOS TESTES")
print("=" * 60)
display(summary_df)

# Estatísticas
total_tests = len(summary_df)
successful_tests = summary_df['success'].sum()
avg_latency = summary_df['latency_ms'].mean()
max_latency = summary_df['latency_ms'].max()
min_latency = summary_df['latency_ms'].min()

print(f"\n📈 ESTATÍSTICAS:")
print(f"  - Testes executados: {total_tests}")
print(f"  - Testes bem-sucedidos: {successful_tests} ({successful_tests/total_tests*100:.1f}%)")
print(f"  - Latência média: {avg_latency:.2f} ms")
print(f"  - Latência mínima: {min_latency:.2f} ms")
print(f"  - Latência máxima: {max_latency:.2f} ms")

In [ ]:
# Gráfico de latências
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Latências por endpoint
summary_df.plot(x='endpoint', y='latency_ms', kind='bar', ax=axes[0], color='steelblue', legend=False)
axes[0].set_title('Latência por Endpoint', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Endpoint')
axes[0].set_ylabel('Latência (ms)')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Taxa de sucesso
success_counts = summary_df['success'].value_counts()
axes[1].pie(success_counts.values, labels=['Sucesso', 'Falha'], autopct='%1.1f%%', 
            colors=['lightgreen', 'lightcoral'], startangle=90)
axes[1].set_title('Taxa de Sucesso dos Testes', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 9️⃣ Análise de Qualidade dos Dados

In [ ]:
print("✅ ANÁLISE DE QUALIDADE DOS DADOS DA API")
print("=" * 60)

quality_checks = []

for test in api_tester.test_results:
    if test.get('success') and 'response' in test:
        data = test['response']
        
        check = {
            'endpoint': test['endpoint'],
            'has_data': len(str(data)) > 0,
            'data_type': type(data).__name__,
            'is_empty': len(data) == 0 if isinstance(data, (list, dict)) else False
        }
        
        # Verificações específicas
        if isinstance(data, list) and len(data) > 0:
            check['count'] = len(data)
            check['sample'] = data[0] if data else None
        elif isinstance(data, dict):
            check['keys'] = list(data.keys())
        
        quality_checks.append(check)

# Exibir análise
for check in quality_checks:
    print(f"\n📋 {check['endpoint']}:")
    print(f"   - Tipo: {check['data_type']}")
    print(f"   - Possui dados: {'✅' if check['has_data'] else '❌'}")
    if 'count' in check:
        print(f"   - Quantidade: {check['count']}")
    if 'keys' in check:
        print(f"   - Chaves: {check['keys']}")

print("\n" + "=" * 60)

## 🔟 Teste de Stress - Múltiplas Requisições

In [ ]:
print("⚡ TESTE DE STRESS - Múltiplas Requisições Sequenciais")
print("=" * 60)

# Fazer 10 requisições seguidas
n_requests = 10
stress_results = []

print(f"\nExecutando {n_requests} requisições...\n")

for i in range(n_requests):
    start = time.time()
    result = api_tester.test_endpoint('Linha/Buscar', params={'termosBusca': 'Centro'})
    elapsed = (time.time() - start) * 1000
    
    stress_results.append({
        'request': i + 1,
        'success': result['success'] if result else False,
        'latency_ms': elapsed
    })
    
    print(f"  Request {i+1}/{n_requests}: {'✅' if result and result['success'] else '❌'} ({elapsed:.2f} ms)")
    time.sleep(0.5)  # Pequeno delay entre requisições

# Análise
stress_df = pd.DataFrame(stress_results)

print(f"\n📊 RESULTADOS DO TESTE DE STRESS:")
print(f"  - Taxa de sucesso: {stress_df['success'].sum()}/{n_requests} ({stress_df['success'].sum()/n_requests*100:.1f}%)")
print(f"  - Latência média: {stress_df['latency_ms'].mean():.2f} ms")
print(f"  - Latência mínima: {stress_df['latency_ms'].min():.2f} ms")
print(f"  - Latência máxima: {stress_df['latency_ms'].max():.2f} ms")
print(f"  - Desvio padrão: {stress_df['latency_ms'].std():.2f} ms")

# Gráfico
plt.figure(figsize=(14, 6))
plt.plot(stress_df['request'], stress_df['latency_ms'], marker='o', linewidth=2, markersize=8)
plt.axhline(y=stress_df['latency_ms'].mean(), color='r', linestyle='--', label=f'Média: {stress_df["latency_ms"].mean():.2f} ms')
plt.title('Latência ao Longo das Requisições - Teste de Stress', fontsize=14, fontweight='bold')
plt.xlabel('Número da Requisição')
plt.ylabel('Latência (ms)')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 📝 Exportar Relatório de Testes

In [ ]:
# Preparar relatório completo
report = {
    'timestamp': datetime.now().isoformat(),
    'summary': {
        'total_tests': total_tests,
        'successful_tests': int(successful_tests),
        'success_rate': float(successful_tests/total_tests*100),
        'avg_latency_ms': float(avg_latency),
        'min_latency_ms': float(min_latency),
        'max_latency_ms': float(max_latency)
    },
    'test_results': api_tester.test_results,
    'stress_test': stress_results,
    'quality_checks': quality_checks
}

# Salvar relatório
report_path = '../data/samples/api_test_report.json'
Path(report_path).parent.mkdir(parents=True, exist_ok=True)

with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False, default=str)

print(f"✅ Relatório salvo em: {report_path}")

# Salvar resumo em CSV
csv_path = '../data/samples/api_test_summary.csv'
summary_df.to_csv(csv_path, index=False)
print(f"✅ Resumo CSV salvo em: {csv_path}")

## 🎯 Conclusões e Recomendações

In [ ]:
print("🎯 CONCLUSÕES E RECOMENDAÇÕES")
print("=" * 60)

conclusions = []

# Análise de performance
if avg_latency < 500:
    conclusions.append("✅ Performance excelente: latência média abaixo de 500ms")
elif avg_latency < 1000:
    conclusions.append("⚠️ Performance boa: latência média entre 500-1000ms")
else:
    conclusions.append("❌ Performance preocupante: latência média acima de 1000ms")

# Análise de confiabilidade
success_rate = (successful_tests/total_tests*100)
if success_rate >= 95:
    conclusions.append("✅ Alta confiabilidade: taxa de sucesso >= 95%")
elif success_rate >= 80:
    conclusions.append("⚠️ Confiabilidade moderada: taxa de sucesso entre 80-95%")
else:
    conclusions.append("❌ Baixa confiabilidade: taxa de sucesso < 80%")

# Recomendações
recommendations = [
    "📌 Implementar retry logic para requisições falhadas",
    "📌 Adicionar circuit breaker para proteção contra falhas",
    "📌 Implementar cache para reduzir carga na API",
    "📌 Monitorar latência e taxa de erro em produção",
    "📌 Considerar rate limiting para evitar sobrecarga"
]

print("\n📊 Conclusões:")
for conclusion in conclusions:
    print(f"  {conclusion}")

print("\n💡 Recomendações:")
for rec in recommendations:
    print(f"  {rec}")

print("\n" + "=" * 60)
print("✅ Testes da API concluídos com sucesso!")